# Import Statements

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint

In [2]:
%matplotlib inline
sns.set_style("darkgrid")

# Load and prepare data

In [3]:
df = pd.read_csv('day.csv', parse_dates=["dteday"])
df = df.drop(["instant", "casual", "registered"], axis=1)
df = df.rename({"dteday": "date"}, axis=1)

In [4]:
df

,date,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,2114
727,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,3095
728,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,1341
729,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,1796


In [5]:
date_column = df.date

df["day_of_year"] = date_column.dt.dayofyear
df["day_of_month"] = date_column.dt.day

df["quarter"] = date_column.dt.quarter
df["week"] = date_column.dt.week

df["is_month_end"] = date_column.dt.is_month_end
df["is_month_start"] = date_column.dt.is_month_start
df["is_quarter_end"] = date_column.dt.is_quarter_end
df["is_quarter_start"] = date_column.dt.is_quarter_start
df["is_year_end"] = date_column.dt.is_year_end
df["is_year_start"] = date_column.dt.is_year_start

df = df.set_index("date")

<ipython-input-5-7f76abfc0f02>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["week"] = date_column.dt.week


In [6]:
df["label"] = df.cnt
df = df.drop("cnt", axis=1)

In [7]:
train_df = df.iloc[:-61]
test_df = df.iloc[-61:]     # Nov and Dec of 2012

In [8]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()

# df['sex'] = encoder.fit_transform(df['sex'])
# df['smoker'] = encoder.fit_transform(df['smoker'])
# df['region'] = encoder.fit_transform(df['region'])

# X = df[df.columns.to_list()[:-1]]
# y = df['charges']
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 121)#, test_size = 0.32)

In [7]:
df = pd.read_csv('insurance.csv')
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['sex'] = encoder.fit_transform(df['sex'])
df['smoker'] = encoder.fit_transform(df['smoker'])
df['region'] = encoder.fit_transform(df['region'])
X = df[df.columns.to_list()[:-1]]
y = df['charges']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 121)#, test_size = 0.32)

In [12]:
tree = DescisionTree(max_depth = 3, criterion = 'mse', ml_task = 'regression')
complete_tree = tree.fit(X_train, y_train) 

from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, tree.predict(X_train)))

4457.0472994953025

In [14]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, tree.predict(X_test)))

5101.821731158357

In [15]:
complete_tree

{'smoker = 1.0 (Node: 1)': [{'bmi <= 30.0 (Node: 2)': [{'age <= 39.0 (Node: 4)': ['18640.563641296296 Node: 6',
      '25857.196740975614 Node: 3']},
    {'age <= 30.0 (Node: 5)': ['36882.46957657896 Node: 8',
      '44431.574282318834 Node: 7']}]},
  {'age <= 44.0 (Node: 9)': [{'children = 0.0 (Node: 10)': ['3888.112809062176 Node: 12',
      '6642.901125830388 Node: 11']},
    {'age <= 51.0 (Node: 13)': ['10295.298627016131 Node: 14',
      '13809.460579850747 Node: 15']}]}]}

# Class

In [3]:
class DescisionTree:
  def __init__(self, criterion,ml_task = 'classification', min_samples_leaf=1, min_samples_split = 2, max_depth = 5):
    self.data = None
    self.X = None
    self.y = None
    self.ml_task = ml_task
    self.min_samples_leaf = min_samples_leaf
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.metric = criterion
    self.feature_importances_ = None
    self.complete_tree = None
    self.n_entries = {}
    self.n_weighted_entries = {}
    self.parent_node = 1  # root node
    self.yes_node = 2     # left node
    self.no_node = 3      # right node
    self.leaf_count = 0
    if ml_task == 'classification': self.classes_and_counts = {};self.leaf_node_class_proba = {}
    else: self.leaf_node_loss = {}
    

  ''' This method is used to get the collective counts of all classes in target '''
  def get_classes_and_counts(self, data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    for i in range(len(unique_classes)):
      self.classes_and_counts[unique_classes[i]] = counts_unique_classes[i]

    
    
    

  ''' This method is used to get the collective probabilities of all classes in target '''
  def get_probability_for_all_classes(self, data):
    label_column = data[:, -1]
    unique_classes_new, counts_unique_classes_new = np.unique(label_column, return_counts=True)
    
    classes_and_counts_new = {}
    for i in list(self.classes_and_counts.keys()):
      if i in list(unique_classes_new):
        classes_and_counts_new[i] = counts_unique_classes_new[list(unique_classes_new).index(i)]
      else:
        classes_and_counts_new[i] = 0
    array = np.array(list(classes_and_counts_new.values())) / sum(classes_and_counts_new.values())

    return [round(i, 5) for i in array]





  ''' This method checks the purity of a target vector '''
  def check_purity(self, data):
      label_column = data[:, -1]
      unique_classes = np.unique(label_column)
      if len(unique_classes) == 1:
          return True
      else:
          return False

    
    
    

  ''' This method performs classification '''
  def create_leaf(self, data, ml_task, current_node):
      self.leaf_count += 1
      label_column = data[:, -1]
      if ml_task == "regression":
          leaf = np.mean(label_column)
          self.leaf_node_loss[current_node] = self.mse(data)
          return str(leaf) + ' Node: '+str(current_node)
      else:
          probabilities = []
          unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
          index = counts_unique_classes.argmax()
          leaf = unique_classes[index]
          # probability = counts_unique_classes[index] / sum(counts_unique_classes)
          self.leaf_node_class_proba[current_node] = self.get_probability_for_all_classes(data)
          
          return str(leaf) + ' Node: '+str(current_node)

    
    
    
  ''' this function generates all possible potential splits for a given training data '''
  def get_potential_splits(self, data):
      potential_splits = {}
      _, n_columns = data.shape
      for column_index in range(n_columns - 1):   
          values = data[:, column_index]
          unique_values = np.unique(values)
          potential_splits[column_index] = unique_values
      
      return potential_splits


    
    
    
  ''' This function splits the data into two partitions: Yes and no cases'''
  def split_data(self, data, split_column, split_value):
      split_column_values = data[:, split_column]
      type_of_feature = FEATURE_TYPES[split_column]

      if type_of_feature == "continuous":
          data_below = data[split_column_values <= split_value]
          data_above = data[split_column_values >  split_value]
      else:
          data_below = data[split_column_values == split_value]
          data_above = data[split_column_values != split_value]
      
      return data_below, data_above


    
    
    
  ''' This method calculates mse loss'''
  def mse(self, data):
      actual_values = data[:, -1]
      if len(actual_values) == 0:   # empty data
          mse = 0
      else:
          prediction = np.mean(actual_values)
          mse = np.mean((actual_values - prediction) **2)
      
      return mse

    
    
    

  ''' This method calculates entropy loss '''
  def entropy(self, data):
      label_column = data[:, -1]
      _, counts = np.unique(label_column, return_counts=True)
      probabilities = counts / counts.sum()
      entropy = sum(probabilities * -np.log2(probabilities))
      
      return entropy


    
    
    
  ''' This method calculates gini impurity'''
  def gini(self, data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()
    gini_index = 0

    for i in probabilities:
      gini_index += i ** 2
    
    return 1 - gini_index





  ''' calculating total/weighed value of the used metric '''
  def calculate_overall_metric(self, data_below, data_above, metric_function):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    # weighted MSE, RMSE, Gini, and entropy
    overall_metric =  (p_data_below * metric_function(self, data_below) 
                     + p_data_above * metric_function(self, data_above))
    
    return overall_metric




  
  ''' Determining which split is the best by using the metric '''
  def determine_best_split(self, data, potential_splits, ml_task, criterion):
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = self.split_data(data, split_column=column_index, split_value=value)
            
        
            current_overall_metric = self.calculate_overall_metric(data_below, data_above, metric_function=criterion)
            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value





  ''' determining the type of a feature among all features '''
  def determine_type_of_feature(self, df):
    feature_types = []
    n_unique_values_treshold = 10

    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

  
    
    
    
  ''' THIS IS THE MAIN RECURSIVE ALGORITHM FOR DESCISION TREE'''
  def tree(self, df, ml_task, counter,current_node, min_samples_leaf, min_samples_split,max_depth, criterion, answer):

    # When the tree starts, the dataframe is converteed to numpy array, the depth of the tree is checked using counter variable and all features types are detected
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = self.determine_type_of_feature(df)
        data = df.values
        criterion = getattr(DescisionTree, criterion)
    else:
        data = df      

    # storing the length of data passed into the node
    self.n_entries['Node: '+str(current_node)] = [len(df)]
    # criterion = getattr(DescisionTree, criterion)

    # storing the loss/mse/rmse/gini/entropy in a specific node
    self.n_entries['Node: '+str(current_node)].append(criterion(self, data))
    
    # incrementing yes/left nodes and no/right nodes such that yes will be a even node and no will be a odd node repectively
    if (answer == 'yes answer'):
      self.yes_node += 2
    elif (answer == 'no answer'):
      self.no_node += 2 
      
    # checking if that target of the data passed is either pure, has minimum samples to create a leaf, or the depth of tree has reached its maximum depth
    if (self.check_purity(data)) or (len(data) == min_samples_leaf) or (counter == max_depth):
        leaf = self.create_leaf(data, ml_task, current_node) # creating the leaf
        return leaf 
    # if above requirements to create a leaf are not met, two new nodes will be created recursively respectively.
    else:    
        counter += 1 # when two new nodes are created, the depth of three is also incremented
        
        # if the data is not yet pure, but has not minimum samples to perform the split, a leaf is created
        if (len(data) < min_samples_split):
          leaf = self.create_leaf(data, ml_task, current_node)
          return leaf
        else:
          # getting the all possible splits, determining which split has least loss, and splitting the data into left and right nodes respectively
          potential_splits = self.get_potential_splits(data)
          split_column, split_value = self.determine_best_split(data, potential_splits, ml_task,criterion)
          data_below, data_above = self.split_data(data, split_column, split_value)
          
          # if the data seperated into left and right nodes, but there is no data, instead of creating a node, a leaft is created
          if len(data_below) == 0 or len(data_above) == 0:
              leaf = self.create_leaf(data, ml_task, current_node)
              return leaf
          
          # finding the type of a selected feature column and its name
          feature_name = COLUMN_HEADERS[split_column]
          type_of_feature = FEATURE_TYPES[split_column]

#           # creating the tree questions
          if type_of_feature == "continuous":
              question = "{} <= {} (Node: {})".format(feature_name, split_value, current_node)
          # feature is categorical
          else:
              question = "{} = {} (Node: {})".format(feature_name, split_value, current_node)

          # instantiate sub-tree
          sub_tree = {question: []}

          # creating left and right nodes recursively
          yes_answer = self.tree(data_below, ml_task, counter, self.yes_node,min_samples_leaf, min_samples_split,max_depth, criterion, 'yes answer')
          no_answer = self.tree(data_above, ml_task, counter, self.no_node,min_samples_leaf,min_samples_split, max_depth, criterion, 'no answer')
          
          # if both left and right nodes are same, only taking one value for a leaf node
          if yes_answer == no_answer:
              sub_tree = yes_answer
          else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
          
          return sub_tree

        
        
        

  ''' making predictions using this tree '''
  def predict_example(self, example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")[:3]

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # base case
    if not isinstance(answer, dict):
        return float(answer.split()[0])
    
    # recursive part
    else:
        residual_tree = answer
        return self.predict_example(example, residual_tree)

    
    
    

  ''' making probability of the predictions using this tree '''
  def predict_example_probability(self, example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")[:3]

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # base case
    if not isinstance(answer, dict):
        return int(answer.split()[2])
            
    # recursive part
    else:
        residual_tree = answer
        return self.predict_example_probability(example, residual_tree)

    
    
    

  ''' fitting our tree with training data '''
  def fit(self, X,y):
    self.X = X.copy();self.y = y.copy()
    self.X['label'] = self.y;self.data = self.X

    if self.ml_task == 'classification':
      self.get_classes_and_counts(self.data.values)
    self.complete_tree = self.tree(self.data, self.ml_task, 0,self.parent_node, self.min_samples_leaf, self.min_samples_split, self.max_depth, self.metric,'parent_node')

    # calculating weighted entries
    for key, value in self.n_entries.items():
      self.n_weighted_entries[key] = [value[0] / len(self.X), value[1]]

    return self.complete_tree





  ''' Making predictions '''
  def predict(self, X):
    # X_train.apply(predict_example, axis=1, args=(tree_gini,))
    predictions = np.array(X.apply(self.predict_example, axis = 1, args = (self.complete_tree, )))
    
    return predictions




  ''' This method returns the predicted probabilities for all classes '''
  def predict_proba(self, X):
    leaf_nodes_for_predictions = np.array(X.apply(self.predict_example_probability, axis = 1, args = (self.complete_tree, )))
    probabilities = [] 
    
    
    for i in leaf_nodes_for_predictions:
      for key, value in self.leaf_node_class_proba.items():
        if i == key:
          probabilities.append(list(value))

    return probabilities

In [10]:
X = train_df[train_df.columns.to_list()[:-1]]
y = train_df['label']

In [11]:
X_test = test_df[test_df.columns.to_list()[:-1]]
y_test = test_df['label']

In [12]:
tree = DescisionTree(max_depth = 3, criterion = 'mse', ml_task = 'regression')
complete_tree = tree.fit(X, y) 

In [13]:
complete_tree

{'temp <= 0.430833 (Node: 1)': [{'yr = 1 (Node: 2)': [{'atemp <= 0.309909 (Node: 4)': ['2938.175 Node: 6',
      '4316.7441860465115 Node: 3']},
    {'season = 4 (Node: 5)': ['3364.734693877551 Node: 8',
      '1711.138888888889 Node: 7']}]},
  {'yr = 1 (Node: 9)': [{'hum <= 0.83125 (Node: 10)': ['6735.924528301887 Node: 12',
      '4203.6 Node: 11']},
    {'weathersit = 3 (Node: 13)': ['2320.5 Node: 14', '4374.32 Node: 15']}]}]}

In [14]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y, tree.predict(X)))

848.5977169927768

In [15]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, tree.predict(X_test)))

1470.923070804387

In [18]:
# 1.  mse garauda kei vaena.. self.mse implement garda
# 2. create leaf ra predit example purai change gare.. additionally, tree ko algorithm bhitra tree string banaune
#    string ni change gae------------ yes


# 3. undo previous changes that did not change any model's performance

In [20]:
# grid_search = {"max_depth": [], "min_samples": [], "r_squared_train": [], "r_squared_val": []}
# for max_depth in range(1, 7):
#     for min_samples in range(5, 20, 5):
#         tree = DescisionTree(max_depth = max_depth, criterion = 'mse', ml_task = 'regression', min_samples_leaf = min_samples)
#         complete_tree = tree.fit(X, y) 
# #         tree = DescisionTree(train_df, ml_task="regression", max_depth=max_depth, min_samples=min_samples, criterion = 'mse')
        
#         r_squared_train = calculate_r_squared(train_df, complete_tree)
#         r_squared_val = calculate_r_squared(test_df, complete_tree)
        
#         grid_search["max_depth"].append(max_depth)
#         grid_search["min_samples"].append(min_samples)
#         grid_search["r_squared_train"].append(r_squared_train)
#         grid_search["r_squared_val"].append(r_squared_val)
        
#     print(f"Progress: Iteration {max_depth}/6")
        
# grid_search = pd.DataFrame(grid_search)
# grid_search.sort_values("r_squared_val", ascending=False).head()

# Classification

In [4]:
df = pd.read_csv('iris.csv')
X = df[df.columns.to_list()[:-1]]
y = df['Species']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 121)#, test_size = 0.32
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)

y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [5]:
tree = DescisionTree(max_depth = 5, criterion = 'gini', ml_task = 'classification')
complete_tree = tree.fit(X_train, y_train) 

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, tree.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91        12
           1       0.89      1.00      0.94        16
           2       1.00      1.00      1.00        10

    accuracy                           0.95        38
   macro avg       0.96      0.94      0.95        38
weighted avg       0.95      0.95      0.95        38



In [10]:
tree.leaf_node_class_proba

{4: [1.0, 0.0, 0.0], 3: [0.0, 1.0, 0.0], 5: [0.0, 0.0, 1.0]}

In [9]:
complete_tree

{'Id <= 100.0 (Node: 1)': [{'PetalWidthCm <= 0.4 (Node: 2)': ['0.0 Node: 4',
    '1.0 Node: 3']},
  '2.0 Node: 5']}

In [14]:
{'Tree_1':tree.predict_proba(X_test)}

{'Tree_1': [[0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [1.0, 0.0, 0.0],
  [1.0, 0.0, 0.0]]}

# SKLEARN

In [23]:
# sklearn tree
from sklearn.tree import DecisionTreeRegressor
tree_ = DecisionTreeRegressor(max_depth = 3, max_features = None)
tree_.fit(X, y)

DecisionTreeRegressor(max_depth=3)

In [24]:
np.sqrt(mean_squared_error(y_test, tree_.predict(X_test)))

1470.923070804387

# extra

In [ ]:
tree = DescisionTree(max_depth = 3, criterion = 'entropy')
complete_tree = tree.fit(X_train, y_train) 

In [ ]:
tree.leaf_node_class_proba

{2: [1.0, 0.0, 0.0],
 5: [0.0, 0.33333, 0.66667],
 6: [0.0, 0.9697, 0.0303],
 7: [0.0, 0.0, 1.0]}

In [ ]:
pprint(complete_tree, width = 20)

{'PetalWidthCm <= 0.4 (Node: 1)': ['0.0 '
                                   'Node: '
                                   '2',
                                   {'PetalWidthCm <= 1.7 (Node: 3)': [{'PetalLengthCm <= 4.9 (Node: 4)': ['1.0 '
                                                                                                          'Node: '
                                                                                                          '6',
                                                                                                          '2.0 '
                                                                                                          'Node: '
                                                                                                          '5']},
                                                                      '2.0 '
                                                                      'Node: '
                                      

In [ ]:
test_predictions = tree.predict(X_test)

In [ ]:
test_predictions_proba = tree.predict_proba(X_test)

[6 7 6 2 2 6 6 6 6 7 7 6 6 2 2 7 6 2 7 2 7 7 6 6 6 6 2 6 7 7 6 7 2 6 6 7 2
 2]


In [ ]:
from sklearn.metrics import roc_auc_score
print('The AUC score measured by one class versus rest classes is: ',roc_auc_score(test_predictions, test_predictions_proba, multi_class = 'ovo'))

The AUC score measured by one class versus rest classes is:  1.0


# Helper Functions

## Data pure?

In [ ]:
''' This function checks the purity of a target vector '''

def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

## Classify

In [ ]:
''' This function performs classification '''

def create_leaf(data, ml_task):
    
    label_column = data[:, -1]
    if ml_task == "regression":
        leaf = np.mean(label_column)
        
    # classfication    
    else:
        unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
        index = counts_unique_classes.argmax()
        leaf = unique_classes[index]
        

    return leaf

## Potential splits?

In [ ]:
''' this function generates all possible potential splits for a given training data '''

def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)

        potential_splits[column_index] = unique_values
        
        # type_of_feature = FEATURE_TYPES[column_index]
        # if type_of_feature == "continuous":
        #     potential_splits[column_index] = []
        #     for index in range(len(unique_values)):
        #         if index != 0:
        #             current_value = unique_values[index]
        #             previous_value = unique_values[index - 1]
        #             potential_split = (current_value + previous_value) / 2

        #             potential_splits[column_index].append(potential_split)
        
        # # feature is categorical
        # # (there need to be at least 2 unique values, otherwise in the
        # # split_data function data_below would contain all data points
        # # and data_above would be empty)
        # elif len(unique_values) > 1:
        #     potential_splits[column_index] = unique_values
    
    return potential_splits

## Split data

In [ ]:
''' This function splits the data into two partitions: Yes and no cases'''

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

## Lowest Overall Entropy or mse?

In [ ]:
## MSE LOSS

def mse(data):
    actual_values = data[:, -1]
    if len(actual_values) == 0:   # empty data
        mse = 0
        
    else:
        prediction = np.mean(actual_values)
        mse = np.mean((actual_values - prediction) **2)
    
    return mse


# Entropy loss
''' Calculating entropy loss '''

def entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [ ]:
# GINI IMPURITY

def gini(data):
  label_column = data[:, -1]
  _, counts = np.unique(label_column, return_counts=True)
  probabilities = counts / counts.sum()
  gini_index = 0

  for i in probabilities:
    gini_index += i ** 2
  
  return 1 - gini_index

In [ ]:
def calculate_overall_metric(data_below, data_above, metric_function):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    # weighted MSE, RMSE, Gini, and entropy
    overall_metric =  (p_data_below * metric_function(data_below) 
                     + p_data_above * metric_function(data_above))
    
    return overall_metric

In [ ]:
def determine_best_split(data, potential_splits, ml_task, criterion):
    
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            
            if ml_task == "regression":
                current_overall_metric = calculate_overall_metric(data_below, data_above, metric_function=criterion)
            
            # classification
            else:
                current_overall_metric = calculate_overall_metric(data_below, data_above, metric_function=criterion)

            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Determine the type of feature

In [ ]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_treshold = 10
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

## Decision Tree Algorithm

In [ ]:
def decision_tree_algorithm(df, ml_task, counter=0, min_samples=2, min_samples_split = 2,max_depth=5, criterion = gini, max_leaf_nodes = 3):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES, current_leaf_node
        current_leaf_node = 0
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values

    else:
        data = df           
    
    # if ()
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        leaf = create_leaf(data, ml_task)
        current_leaf_node += 1
        return leaf
 
    # recursive part
    else:    
        counter += 1
        if (len(data) < min_samples_split):
          leaf = create_leaf(data, ml_task)
          current_leaf_node += 1
        else:
          # helper functions 
          potential_splits = get_potential_splits(data)
          split_column, split_value = determine_best_split(data, potential_splits, ml_task,criterion = criterion)
          data_below, data_above = split_data(data, split_column, split_value)
          
          # check for empty data
          if len(data_below) == 0 or len(data_above) == 0:
              leaf = create_leaf(data, ml_task)
              current_leaf_node += 1
              return leaf
          
          # determine question
          feature_name = COLUMN_HEADERS[split_column]
          type_of_feature = FEATURE_TYPES[split_column]
          if type_of_feature == "continuous":
              question = "{} <= {}".format(feature_name, split_value)
              
          # feature is categorical
          else:
              question = "{} = {}".format(feature_name, split_value)
          
          # instantiate sub-tree
          sub_tree = {question: []}
          
          # find answers (recursion)
          yes_answer = decision_tree_algorithm(data_below, ml_task, counter, min_samples, min_samples_split,max_depth, criterion)
          no_answer = decision_tree_algorithm(data_above, ml_task, counter, min_samples,min_samples_split, max_depth, criterion)
          
          # If the answers are the same, then there is no point in asking the qestion.
          # This could happen when the data is classified even though it is not pure
          # yet (min_samples or max_depth base case).
            
          if yes_answer == no_answer:
              sub_tree = yes_answer
          else:
              sub_tree[question].append(yes_answer)
              sub_tree[question].append(no_answer)
          
          
          return sub_tree

In [ ]:
X_train['label'] = y_train

In [ ]:
%%time
tree_gini = decision_tree_algorithm(X_train, max_depth=5, ml_task="classification", criterion = gini)

CPU times: user 66.8 ms, sys: 11.6 ms, total: 78.3 ms
Wall time: 73.7 ms


In [ ]:
pprint(tree_gini)

{'PetalWidthCm <= 0.4': ['Iris-setosa',
                         {'PetalWidthCm <= 1.7': [{'PetalLengthCm <= 4.9': [{'PetalWidthCm <= 1.6': ['Iris-versicolor',
                                                                                                     'Iris-virginica']},
                                                                            {'PetalWidthCm <= 1.5': ['Iris-virginica',
                                                                                                     {'PetalLengthCm <= 5.1': ['Iris-versicolor',
                                                                                                                               'Iris-virginica']}]}]},
                                                  'Iris-virginica']}]}


In [ ]:
pprint(complete_tree)

{'PetalWidthCm <= 0.4 (Node: 1)': ['0.0 Node: 2',
                                   {'PetalWidthCm <= 1.7 (Node: 3)': [{'PetalLengthCm <= 4.9 (Node: 4)': [{'PetalWidthCm <= 1.6 (Node: 6)': ['1.0 '
                                                                                                                                             'Node: '
                                                                                                                                             '8',
                                                                                                                                             '2.0 '
                                                                                                                                             'Node: '
                                                                                                                                             '5']},
                                                            

In [ ]:
current_leaf_node

7

In [ ]:
%%time
tree_entropy = decision_tree_algorithm(X_train, max_depth=5, ml_task="classification", criterion = entropy)

CPU times: user 120 ms, sys: 16.1 ms, total: 136 ms
Wall time: 124 ms


## Classification

In [ ]:
def predict_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

In [ ]:
tree

In [ ]:
X_train.apply(predict_example, axis=1, args=(tree_gini,))

20         Iris-setosa
134     Iris-virginica
123     Iris-virginica
138     Iris-virginica
137     Iris-virginica
            ...       
83     Iris-versicolor
65     Iris-versicolor
95     Iris-versicolor
8          Iris-setosa
66     Iris-versicolor
Length: 112, dtype: object

In [ ]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(predict_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

X_test['label'] = y_test

print("For training data")
print("tree_gini: ", calculate_accuracy(X_train, tree_gini))
print("tree_entropy: ", calculate_accuracy(X_train, tree_entropy))

print("\nFor testing data")
print("tree_gini: ", calculate_accuracy(X_test, tree_gini))
print("tree_entropy: ", calculate_accuracy(X_test, tree_entropy))

For training data
tree_gini:  1.0
tree_entropy:  1.0

For testing data
tree_gini:  0.9210526315789473
tree_entropy:  0.9210526315789473


# Post pruning

In [ ]:
def predict_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

In [ ]:
# 3.2 All examples of a dataframe
def make_predictions(df, tree):
    
    if len(df) != 0:
        predictions = df.apply(predict_example, args=(tree,), axis=1)
    else:
        # "df.apply()"" with empty dataframe returns an empty dataframe,
        # but "predictions" should be a series instead
        predictions = pd.Series()
        
    return predictions

In [ ]:
def filter_df(df, question):
    feature, comparison_operator, value = question.split()
    
    # continuous feature
    if comparison_operator == "<=":
        df_yes = df[df[feature] <= float(value)]
        df_no =  df[df[feature] >  float(value)]
        
    # categorical feature
    else:
        df_yes = df[df[feature].astype(str) == value]
        df_no  = df[df[feature].astype(str) != value]
    
    return df_yes, df_no

In [ ]:
def determine_leaf(df_train, ml_task):
    
    if ml_task == "regression":
        return df_train.label.mean()
    
    # classification
    else:
        return df_train.label.value_counts().index[0]

In [ ]:
def determine_traning_mode_or_mean_errors(df_val, leaf, ml_task):
    predictions = leaf
    actual_values = df_val.label
    
    if ml_task == "regression":
        # mean squared error
        return ((predictions - actual_values) **2).mean()
    else:
        # number of errors
        return sum(predictions != actual_values)

In [ ]:
def determine_descision_node_errors(df_val, tree, ml_task):
    predictions = make_predictions(df_val, tree)
    actual_values = df_val.label
    
    if ml_task == "regression":
        # mean squared error
        return ((predictions - actual_values) **2).mean()
    else:
        # number of errors
        return sum(predictions != actual_values)

In [ ]:
def pruning_result(tree, df_train, df_val, ml_task):
    
    leaf = determine_leaf(df_train, ml_task)
    errors_leaf = determine_traning_mode_or_mean_errors(df_val, leaf, ml_task) # training data ko mode/mean use garera validation ma nikalney error
    errors_decision_node = determine_descision_node_errors(df_val, tree, ml_task) # actual validation error

    if errors_leaf <= errors_decision_node:
        return leaf
    else:
        return tree

In [ ]:
def post_pruning(tree, df_train, df_val, ml_task):
    
    question = list(tree.keys())[0]
    yes_answer, no_answer = tree[question]

    # base case
    if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
        return pruning_result(tree, df_train, df_val, ml_task)
        
    # recursive part
    else:
        df_train_yes, df_train_no = filter_df(df_train, question)
        df_val_yes, df_val_no = filter_df(df_val, question)
        
        if isinstance(yes_answer, dict):
            yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes, ml_task)
            
        if isinstance(no_answer, dict):
            no_answer = post_pruning(no_answer, df_train_no, df_val_no, ml_task)
        
        tree = {question: [yes_answer, no_answer]}
    
        return pruning_result(tree, df_train, df_val, ml_task)

In [ ]:
pprint(complete_tree)

{'PetalWidthCm <= 0.4 (Node: 1)': ['0.0 Node: 2',
                                   {'PetalWidthCm <= 1.7 (Node: 3)': [{'PetalLengthCm <= 4.9 (Node: 4)': [{'PetalWidthCm <= 1.6 (Node: 6)': ['1.0 '
                                                                                                                                             'Node: '
                                                                                                                                             '8',
                                                                                                                                             '2.0 '
                                                                                                                                             'Node: '
                                                                                                                                             '5']},
                                                            

In [ ]:
X_test['label'] = y_test

In [ ]:
pruned_tree = post_pruning(complete_tree, X_train, X_test, ml_task = 'classification')

AttributeError: ignored

In [ ]:
pruned_tree

{'PetalWidthCm <= 0.6': ['Iris-setosa',
  {'PetalLengthCm <= 4.7': ['Iris-versicolor',
    {'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.7': ['Iris-versicolor',
        'Iris-virginica']},
      'Iris-virginica']}]}]}

In [ ]:
calculate_accuracy(X_test, tree)

0.9473684210526315

In [ ]:
calculate_accuracy(X_test, pruned_tree)

1.0

In [ ]:
pprint(tree)

{'PetalWidthCm <= 0.6': ['Iris-setosa',
                         {'PetalLengthCm <= 4.7': [{'PetalWidthCm <= 1.6': ['Iris-versicolor',
                                                                            'Iris-virginica']},
                                                   {'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.7': [{'SepalWidthCm <= 2.2': ['Iris-virginica',
                                                                                                                               {'PetalLengthCm <= 5.0': ['Iris-versicolor',
                                                                                                                                                         {'PetalWidthCm <= 1.5': ['Iris-virginica',
                                                                                                                                                                                  'Iris-versicolor']}]}]},
                                                     

In [ ]:
pprint(pruned_tree)

{'PetalWidthCm <= 0.6': ['Iris-setosa',
                         {'PetalLengthCm <= 4.7': ['Iris-versicolor',
                                                   {'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.7': ['Iris-versicolor',
                                                                                                      'Iris-virginica']},
                                                                             'Iris-virginica']}]}]}


# Final Codew

In [ ]:
class DescisionTree:
  def __init__(self, criterion,ml_task = 'classification', min_samples_leaf=1, min_samples_split = 2, max_depth = 5):
    self.data = None
    self.X = None
    self.y = None
    self.ml_task = ml_task
    self.min_samples_leaf = min_samples_leaf
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.metric = criterion
    self.feature_importances_ = None
    self.complete_tree = None
    self.n_entries = {}
    self.n_weighted_entries = {}
    self.parent_node = 1  # root node
    self.yes_node = 2     # left node
    self.no_node = 3      # right node
    self.leaf_count = 0
    if ml_task == 'classification': self.classes_and_counts = {};self.leaf_node_class_proba = {}
    else: self.leaf_node_loss = {}
    

  ''' This method is used to get the collective counts of all classes in target '''
  def get_classes_and_counts(self, data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    for i in range(len(unique_classes)):
      self.classes_and_counts[unique_classes[i]] = counts_unique_classes[i]

    
    
    

  ''' This method is used to get the collective probabilities of all classes in target '''
  def get_probability_for_all_classes(self, data):
    label_column = data[:, -1]
    unique_classes_new, counts_unique_classes_new = np.unique(label_column, return_counts=True)
    
    classes_and_counts_new = {}
    for i in list(self.classes_and_counts.keys()):
      if i in list(unique_classes_new):
        classes_and_counts_new[i] = counts_unique_classes_new[list(unique_classes_new).index(i)]
      else:
        classes_and_counts_new[i] = 0
    array = np.array(list(classes_and_counts_new.values())) / sum(classes_and_counts_new.values())

    return [round(i, 5) for i in array]





  ''' This method checks the purity of a target vector '''
  def check_purity(self, data):
      label_column = data[:, -1]
      unique_classes = np.unique(label_column)
      if len(unique_classes) == 1:
          return True
      else:
          return False

    
    
    

  ''' This method performs classification '''
  def create_leaf(self, data, ml_task, current_node):
      self.leaf_count += 1
      label_column = data[:, -1]
      if ml_task == "regression":
          leaf = np.mean(label_column)
          self.leaf_node_loss[current_node] = self.mse(data)
          return str(leaf) + ' Node: '+str(current_node)
      else:
          probabilities = []
          unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
          index = counts_unique_classes.argmax()
          leaf = unique_classes[index]
          # probability = counts_unique_classes[index] / sum(counts_unique_classes)
          self.leaf_node_class_proba[current_node] = self.get_probability_for_all_classes(data)
          
          return str(leaf) + ' Node: '+str(current_node)

    
    
    
  ''' this function generates all possible potential splits for a given training data '''
  def get_potential_splits(self, data):
      potential_splits = {}
      _, n_columns = data.shape
      for column_index in range(n_columns - 1):   
          values = data[:, column_index]
          unique_values = np.unique(values)
          potential_splits[column_index] = unique_values
      
      return potential_splits


    
    
    
  ''' This function splits the data into two partitions: Yes and no cases'''
  def split_data(self, data, split_column, split_value):
      split_column_values = data[:, split_column]
      type_of_feature = FEATURE_TYPES[split_column]

      if type_of_feature == "continuous":
          data_below = data[split_column_values <= split_value]
          data_above = data[split_column_values >  split_value]
      else:
          data_below = data[split_column_values == split_value]
          data_above = data[split_column_values != split_value]
      
      return data_below, data_above


    
    
    
  ''' This method calculates mse loss'''
  def mse(self, data):
      actual_values = data[:, -1]
      if len(actual_values) == 0:   # empty data
          mse = 0
      else:
          prediction = np.mean(actual_values)
          mse = np.mean((actual_values - prediction) **2)
      
      return mse

    
    
    

  ''' This method calculates entropy loss '''
  def entropy(self, data):
      label_column = data[:, -1]
      _, counts = np.unique(label_column, return_counts=True)
      probabilities = counts / counts.sum()
      entropy = sum(probabilities * -np.log2(probabilities))
      
      return entropy


    
    
    
  ''' This method calculates gini impurity'''
  def gini(self, data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()
    gini_index = 0

    for i in probabilities:
      gini_index += i ** 2
    
    return 1 - gini_index





  ''' calculating total/weighed value of the used metric '''
  def calculate_overall_metric(self, data_below, data_above, metric_function):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    # weighted MSE, RMSE, Gini, and entropy
    overall_metric =  (p_data_below * metric_function(self, data_below) 
                     + p_data_above * metric_function(self, data_above))
    
    return overall_metric




  
  ''' Determining which split is the best by using the metric '''
  def determine_best_split(self, data, potential_splits, ml_task, criterion):
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = self.split_data(data, split_column=column_index, split_value=value)
            
        
            current_overall_metric = self.calculate_overall_metric(data_below, data_above, metric_function=criterion)
            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value





  ''' determining the type of a feature among all features '''
  def determine_type_of_feature(self, df):
    feature_types = []
    n_unique_values_treshold = 10

    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

  
    
    
    
  ''' THIS IS THE MAIN RECURSIVE ALGORITHM FOR DESCISION TREE'''
  def tree(self, df, ml_task, counter,current_node, min_samples_leaf, min_samples_split,max_depth, criterion, answer):

    # When the tree starts, the dataframe is converteed to numpy array, the depth of the tree is checked using counter variable and all features types are detected
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = self.determine_type_of_feature(df)
        data = df.values
        criterion = getattr(DescisionTree, criterion)
    else:
        data = df      

    # storing the length of data passed into the node
    self.n_entries['Node: '+str(current_node)] = [len(df)]
    # criterion = getattr(DescisionTree, criterion)

    # storing the loss/mse/rmse/gini/entropy in a specific node
    self.n_entries['Node: '+str(current_node)].append(criterion(self, data))
    
    # incrementing yes/left nodes and no/right nodes such that yes will be a even node and no will be a odd node repectively
    if (answer == 'yes answer'):
      self.yes_node += 2
    elif (answer == 'no answer'):
      self.no_node += 2 
      
    # checking if that target of the data passed is either pure, has minimum samples to create a leaf, or the depth of tree has reached its maximum depth
    if (self.check_purity(data)) or (len(data) == min_samples_leaf) or (counter == max_depth):
        leaf = self.create_leaf(data, ml_task, current_node) # creating the leaf
        return leaf 
    # if above requirements to create a leaf are not met, two new nodes will be created recursively respectively.
    else:    
        counter += 1 # when two new nodes are created, the depth of three is also incremented
        
        # if the data is not yet pure, but has not minimum samples to perform the split, a leaf is created
        if (len(data) < min_samples_split):
          leaf = self.create_leaf(data, ml_task, current_node)
          return leaf
        else:
          # getting the all possible splits, determining which split has least loss, and splitting the data into left and right nodes respectively
          potential_splits = self.get_potential_splits(data)
          split_column, split_value = self.determine_best_split(data, potential_splits, ml_task,criterion)
          data_below, data_above = self.split_data(data, split_column, split_value)
          
          # if the data seperated into left and right nodes, but there is no data, instead of creating a node, a leaft is created
          if len(data_below) == 0 or len(data_above) == 0:
              leaf = self.create_leaf(data, ml_task, current_node)
              return leaf
          
          # finding the type of a selected feature column and its name
          feature_name = COLUMN_HEADERS[split_column]
          type_of_feature = FEATURE_TYPES[split_column]

#           # creating the tree questions
          if type_of_feature == "continuous":
              question = "{} <= {} (Node: {})".format(feature_name, split_value, current_node)
          # feature is categorical
          else:
              question = "{} = {} (Node: {})".format(feature_name, split_value, current_node)

          # instantiate sub-tree
          sub_tree = {question: []}

          # creating left and right nodes recursively
          yes_answer = self.tree(data_below, ml_task, counter, self.yes_node,min_samples_leaf, min_samples_split,max_depth, criterion, 'yes answer')
          no_answer = self.tree(data_above, ml_task, counter, self.no_node,min_samples_leaf,min_samples_split, max_depth, criterion, 'no answer')
          
          # if both left and right nodes are same, only taking one value for a leaf node
          if yes_answer == no_answer:
              sub_tree = yes_answer
          else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
          
          return sub_tree

        
        
        

  ''' making predictions using this tree '''
  def predict_example(self, example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")[:3]

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # base case
    if not isinstance(answer, dict):
        return float(answer.split()[0])
    
    # recursive part
    else:
        residual_tree = answer
        return self.predict_example(example, residual_tree)

    
    
    

  ''' making probability of the predictions using this tree '''
  def predict_example_probability(self, example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")[:3]

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    # base case
    if not isinstance(answer, dict):
        return int(answer.split()[2])
            
    # recursive part
    else:
        residual_tree = answer
        return self.predict_example_probability(example, residual_tree)

    
    
    

  ''' fitting our tree with training data '''
  def fit(self, X,y):
    self.X = X.copy();self.y = y.copy()
    self.X['label'] = self.y;self.data = self.X

    if self.ml_task == 'classification':
      self.get_classes_and_counts(self.data.values)
    self.complete_tree = self.tree(self.data, self.ml_task, 0,self.parent_node, self.min_samples_leaf, self.min_samples_split, self.max_depth, self.metric,'parent_node')

    # calculating weighted entries
    for key, value in self.n_entries.items():
      self.n_weighted_entries[key] = [value[0] / len(self.X), value[1]]

    return self.complete_tree





  ''' Making predictions '''
  def predict(self, X):
    # X_train.apply(predict_example, axis=1, args=(tree_gini,))
    predictions = np.array(X.apply(self.predict_example, axis = 1, args = (self.complete_tree, )))
    
    return predictions




  ''' This method returns the predicted probabilities for all classes '''
  def predict_proba(self, X):
    leaf_nodes_for_predictions = np.array(X.apply(self.predict_example_probability, axis = 1, args = (self.complete_tree, )))
    probabilities = [] 
    
    
    for i in leaf_nodes_for_predictions:
      for key, value in self.leaf_node_class_proba.items():
        if i == key:
          probabilities.append(list(value))

    return probabilities